# Welcome to ProgPy's LSTM Model Tutorial

In this example, we generate fake data using the ThrownObject model. This is a case where we're generating a surrogate model from the physics-based model. For cases where you're generating a model from data (e.g., collected from a testbed or a real-world environment), you'll replace that generated data with your own. We then use the generated model and compare to the original model.

Finally, we repeat the exercise with data from the more complex BatteryElectroChemEOD model.

A State Transition Model with no `event` using an Keras LSTM Model.
State transition models map from the `input` at time t and `output` at time t-1 plus historical data from a set window to the `output` at time t.

Most users will use the :py:func:`LSTMStateTransitionModel.from_data` method to create a model, but the model can be created by passing in a model directly into the constructor. The LSTM model in this method maps from [u_t-n+1, z_t-n, ..., u_t, z_t-1] to z_t. Past `input` are stored in the `model` internal `state`. Actual calculation of `output` is performed when `LSTMStateTransitionModel.output` is called. When using in simulation that may not be until the simulation results are accessed.

However, before we can talk about a State Transition Model, we must describe a data_model !

Abstract Base Class for all Data Models (e.g., `LSTMStateTransitionModel`). Defines the interface and all common tools. To create a new Data-Driven model, first subclass this, then define the abstract methods from this class and `prog_models.PrognosticsModel`!